In [27]:
import requests
import json

In [28]:
GET_URL = "https://candidate.hubteam.com/candidateTest/v3/problem/dataset?userKey=6992c4b8565a290df5bcce80d123"
RESULT_URL = "https://candidate.hubteam.com/candidateTest/v3/problem/result?userKey=6992c4b8565a290df5bcce80d123"
SESSION_LENGTH = 600000

In [29]:
response = requests.get(GET_URL)

In [30]:
events = response.json()['events']
events[0]

{'url': '/punish-elfin-temper',
 'visitorId': '19498f4c-3546-32f2-9fed-ad675aad5175',
 'timestamp': 1515042103034}

In [31]:
events_dict = {}
for event in events:
    id = event['visitorId']
    if id not in events_dict:
        events_dict[id] = []
    events_dict[id].append(event)

In [32]:
events_dict['19498f4c-3546-32f2-9fed-ad675aad5175'][0]['timestamp']

1515042103034

I actually basically had this code on hand from code where we've been .... calculating session in our chatbot. So... just need to rewrite to match the data structure and add durations.

In [33]:
def process_user(events):
    sessions = []
    events.sort(key=lambda x: x['timestamp'])
    
    start = events[0]['timestamp']
    last_event_time = start
    current_session = []
    
    for event in events:
        if event['timestamp'] - last_event_time <= SESSION_LENGTH:
            current_session.append(event)
            last_event_time = event['timestamp']
        else:
            pages = []
            for x in current_session:
                pages.append(x['url'])

            sessions.append({
                "duration": last_event_time - start,
                "pages": pages,
                "startTime": start
            })
            # Re-start the session
            current_session = [event]
            start = event['timestamp']
            last_event_time = event['timestamp'] 
            
    #The deadly last loop you forget about (Ask me about our gpt3-divinci DAG from 2022 that took 10 days to run)
    if current_session:
            pages = []
            for x in current_session:
                pages.append(x['url'])

            sessions.append({
                "duration": last_event_time - start,
                "pages": pages,
                "startTime": start
            })
    
    return sessions


In [34]:
sessions_by_user = {}
for visitor, events in events_dict.items():
    sessions_by_user[visitor] = process_user(events)

In [35]:
print(json.dumps(sessions_by_user, indent=2))

{
  "19498f4c-3546-32f2-9fed-ad675aad5175": [
    {
      "duration": 0,
      "pages": [
        "/punish-elfin-temper"
      ],
      "startTime": 1515042103034
    },
    {
      "duration": 0,
      "pages": [
        "/gaze-panoramic-wealth/procure-wretched-afternoon"
      ],
      "startTime": 1515063757915
    },
    {
      "duration": 0,
      "pages": [
        "/divert-aromatic-apparatus/rejoice-foolish-bedroom"
      ],
      "startTime": 1515065216554
    },
    {
      "duration": 0,
      "pages": [
        "/battle-humdrum-view"
      ],
      "startTime": 1515065915818
    },
    {
      "duration": 153956,
      "pages": [
        "/mix-scandalous-clam/promote-lackadaisical-rub",
        "/generate-fanatical-frogs"
      ],
      "startTime": 1515068447540
    },
    {
      "duration": 0,
      "pages": [
        "/suit-functional-linen/reduce-scarce-pot"
      ],
      "startTime": 1515075608996
    },
    {
      "duration": 0,
      "pages": [
        "/note-attr

Now, let's post it!

In [36]:
result = {
    "sessionsByUser": sessions_by_user
}
response = requests.post(RESULT_URL, json=result)

In [37]:
print(response.status_code)
print(response.content)

200
b'{"message":"Results match! Congratulations!"}'
